In [ ]:
!nvidia-smi

Wed Nov 25 05:17:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing
ResUNetplusplus


In [ ]:
%cd ./ResUNetplusplus
!ls 

/content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing/ResUNetplusplus
'Copy of execution_script.ipynb'   process_image_sample.py
 data				   __pycache__
 data_generator.py		   README.md
 deployment_model.py		   result
 execution_script.ipynb		   results.png
 files				   resume_training.py
 frozen_models			   Resunet++.png
 GFLOPs.ipynb			   resunet_pp_modified.py
 infer.py			   resunet.py
 metrics.py			   run_miouTesting.py
 m_resunet_modified.py		   run_modified.py
 m_resunet.py			   run.py
 new_data			   run_sample.py
 process_image_deployment.py	   unet.py
 process_image.py


In [ ]:
# %pycat process_image.py
# %%writefile process_image.py
!python3 process_image.py #- to augment training dataset.
# !python3 data_generator.py
# Total Size:  1000
# Training Size:  800
# Validation Size:  100
# Testing Size:  100
# 100% 100/100 [00:04<00:00, 24.42it/s]
# 100% 100/100 [00:04<00:00, 22.95it/s]
# 100% 800/800 [53:22<00:00,  4.00s/it]

Total Size:  1000
Training Size:  800
Validation Size:  100
Testing Size:  100
100% 100/100 [00:04<00:00, 23.72it/s]
100% 100/100 [00:04<00:00, 22.00it/s]
100% 800/800 [37:00<00:00,  2.78s/it]


In [ ]:
# Sepv_Conv
# %pycat run_modified.py
# %%writefile run_modified.py
!python3 run_modified.py

2020-11-23 09:47:25.778409: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
train steps: 3100 valid_steps 12
2020-11-23 09:47:30.527863: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-23 09:47:30.566846: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-23 09:47:30.567476: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-11-23 09:47:30.567526: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-23 09:47:30.792892

In [ ]:
# %pycat resume_training.py
# %%writefile resume_training.py
!python3 resume_training.py

2020-11-24 07:54:56.739917: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
train steps: 3100 valid_steps 12
2020-11-24 07:54:59.569913: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-24 07:54:59.584067: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-24 07:54:59.585023: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-11-24 07:54:59.585079: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-24 07:54:59.586841

In [ ]:
# %pycat infer.py
!python3 infer.py

2020-11-25 05:19:18.076813: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-25 05:19:27.938502: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-25 05:19:28.004390: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-25 05:19:28.005031: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-11-25 05:19:28.005099: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-25 05:19:28.263165: I tensorflow/stream_executor/platform/default

In [ ]:
# %pycat run_modified.py
# %%writefile run_modified.py
!python3 run_modified.py

2020-11-24 06:48:35.437075: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
train steps: 3100 valid_steps 12
2020-11-24 06:48:38.091964: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-24 06:48:38.112192: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-24 06:48:38.113151: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-11-24 06:48:38.113202: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-24 06:48:38.115614

In [ ]:
# %pycat run_modified.py
# %%writefile run_modified.py
!python3 run_modified.py

2020-10-17 07:18:20.721286: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
train steps: 3100 valid_steps 12
2020-10-17 07:18:25.260880: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-17 07:18:25.295058: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-17 07:18:25.295745: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-10-17 07:18:25.295800: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-17 07:18:25.502955

In [ ]:
# %pycat infer.py
# dsapp_miou
# 100/100 [==============================] - 3s 28ms/step - loss: 0.3329 - recall: 0.6972 - precision: 0.8262 - dice_coef: 0.7666 - mean_io_u: 0.7529 - miou_coef: 0.6671
# 100%|██████████| 100/100 [00:09<00:00, 10.07it/s]

# miou
# 100/100 [==============================] - 3s 28ms/step - loss: 0.3052 - recall: 0.6497 - precision: 0.8981 - dice_coef: 0.7835 - mean_io_u: 0.7310 - miou_coef: 0.6948
# 100%|██████████| 100/100 [00:10<00:00,  9.93it/s]

# dsapp_relu_miou
# 100/100 [==============================] - 3s 30ms/step - loss: 0.3397 - recall: 0.6154 - precision: 0.9233 - dice_coef: 0.7463 - mean_io_u: 0.7287 - miou_coef: 0.6603
# 100%|██████████| 100/100 [00:48<00:00,  2.08it/s]

# sepv_conv_miou 10 epochs
# 100/100 [==============================] - 2s 23ms/step - loss: 0.3244 - recall: 0.6055 - precision: 0.9331 - dice_coef: 0.7725 - mean_io_u: 0.6793 - miou_coef: 0.6756
# 100% 100/100 [01:05<00:00,  1.53it/s]

# sepv_conv_miou 10+12 epochs
# 100/100 [==============================] - 40s 400ms/step - loss: 0.2878 - recall: 0.6073 - precision: 0.9337 - dice_coef: 0.8003 - mean_io_u: 0.7112 - miou_coef: 0.7122
# 100% 100/100 [00:44<00:00,  2.22it/s]

## dsapp_relu_miou_25
# 100/100 [==============================] - 50s 500ms/step - loss: 0.3111 - recall: 0.6371 - precision: 0.9483 - dice_coef: 0.7703 - mean_io_u: 0.7516 - miou_coef: 0.6889
# 100%|██████████| 100/100 [00:33<00:00,  2.96it/s]

## Unet
# 100/100 [==============================] - 38s 382ms/step - loss: 0.4047 - recall: 0.7523 - precision: 0.8452 - dice_coef: 0.7191 - mean_io_u: 0.4246 - miou_coef: 0.5953

In [ ]:
## GFLOPs
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope
from data_generator import *
from metrics import dice_coef, dice_loss, miou_coef, miou_loss

def get_flops(model_h5_path):
    session = tf.compat.v1.Session()
    graph = tf.compat.v1.get_default_graph()


    with graph.as_default():
        with session.as_default():
          with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef,'miou_loss':miou_loss,'miou_coef':miou_coef}):
            model = load_model(model_h5_path)
            # model = tf.keras.models.load_model(model_h5_path)

          run_meta = tf.compat.v1.RunMetadata()
          opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

          # Optional: save printed results to file
          # flops_log_path = os.path.join(tempfile.gettempdir(), 'tf_flops_log.txt')
          # opts['output'] = 'file:outfile={}'.format(flops_log_path)

          # We use the Keras session graph in the call to the profiler.
          flops = tf.compat.v1.profiler.profile(graph=graph,
                                                run_meta=run_meta, cmd='op', options=opts)
          print(model.summary())
          return flops.total_float_ops

In [ ]:
# model_path = "files/daspp_relu_miou.h5" get_flops = 30074652
# model_path = "files/daspp_miou.h5" get_flops = 40099550
# model_path = "files/miou.h5" get_flops = 8718068
model_path = "files/miou.h5"
get_flops(model_path)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 16) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

17436136

In [ ]:
# %%writefile resume_training.py?
!python3 resume_training.py

2020-10-21 09:47:53.990083: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
train steps: 3100 valid_steps 12
2020-10-21 09:47:56.828144: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-21 09:47:56.842161: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-21 09:47:56.843208: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-10-21 09:47:56.843280: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-21 09:47:56.845674

In [ ]:
!python3 deployment_model.py

2020-10-26 04:57:42.787415: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
train steps: 3487 valid_steps 12
2020-10-26 04:57:47.091953: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-26 04:57:47.148061: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-26 04:57:47.148761: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-10-26 04:57:47.148837: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-26 04:57:47.376242: I tensorflow

In [ ]:
## Submission 

import os
import time
import numpy as np
import tensorflow as tf
import cv2
import pandas as pd
from tensorflow.keras.utils import CustomObjectScope
from metrics import dice_coef, dice_loss, miou_coef, miou_loss

%cd ./new_data/

# Set path to test dataset
TEST_DATASET_PATH = "./test_images"
MASK_PATH = "./mask"

# Load Keras model
# model = tf.keras.models.load_model("../files/daspp_relu_deployment.h5")
with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef,'miou_loss':miou_loss,'miou_coef':miou_coef}):
  model = tf.keras.models.load_model("../files/daspp_miou.h5")

time_taken = []
for image_name in os.listdir(TEST_DATASET_PATH):

    # Load the test image
    image_path = os.path.join(TEST_DATASET_PATH, image_name)
    image = cv2.imread(image_path)
    H, W, _ = image.shape
    image = cv2.resize(image, (512, 512))
    image = np.expand_dims(image, axis=0)

    # Start time
    start_time = time.time()

    ## Prediction
    mask = model.predict(image)[0]

    # End timer
    end_time = time.time() - start_time

    time_taken.append(end_time)
    print("{} - {:.10f}".format(image_name, end_time))

    mask = mask > 0.5
    mask = mask.astype(np.float32)
    mask = mask * 255.0
    mask = cv2.resize(mask, (H, W))

    mask_path = os.path.join(MASK_PATH, image_name)
    cv2.imwrite(mask_path, mask)

mean_time_taken = np.mean(time_taken)
mean_fps = 1/mean_time_taken
print("Mean FPS: ", mean_fps)


/content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing/ResUNetplusplus/new_data
ckcbm9wu21iag0y7m64rm6oz7.jpg - 1.1992387772
ckcbm7flj1fid0y6gcsmoeipg.jpg - 0.1188011169
ckcbm6dni1fcc0y6ggt5c7hlb.jpg - 0.1176197529
ckcbm3tjn0dyu0y4h4kbld9ro.jpg - 0.1112661362
ckcblwsw91koh0y5pe2pt84hl.jpg - 0.0937995911
ckcblygox1fjn0y5n4t9w76xr.jpg - 0.0965981483
ckcblv2mg0cnv0y6h6g71f1wm.jpg - 0.0932717323
ckcbltunv1g4g0y7m84o5fvix.jpg - 0.0944275856
ckcblrxjs1dbr0y6g0i0qgy3g.jpg - 0.1022121906
ckcblq99h0bzq0y4hddj432v9.jpg - 0.0895729065
ckcblngeo1jai0y5p6psactck.jpg - 0.1060869694
ckcbkz2hz1avm0y5n69y6c3hc.jpg - 0.0897381306
ckcblfowe0abp0y4hf95z4q6o.jpg - 0.0918624401
ckcbkx23f1bvd0y7m823u8lb6.jpg - 0.0935528278
ckcblj30z1ei40y7m1w963ryj.jpg - 0.0888483524
ckcbkgdfw1djg0y5p7qu9bmcy.jpg - 0.0877277851
ckcbkvuh01agn0y5n2sqs4vhy.jpg - 0.0903928280
ckcaidxum2xrf0y6g0329hw80.jpg - 0.0997719765
ckcaibt572pxu0y6fazrxcc4f.jpg - 0.0871155262
ckcai9nma2xig0y6g595c7mtj.jpg 

In [ ]:
mask_path

'./mask/ckda4xclk001a3a5svnxcascb.jpg'

In [ ]:
!ls

Kvasir-SEG  Kvasir-SEG_deployment  Kvasir-SEG_tested_correct  test_images
